In [1]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.8.3


In [2]:
BATCH_SIZE = 32 
# classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# (training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\kyley\Desktop\CS\C147\C147 Amazon Classification\img_uniform_split/train", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=BATCH_SIZE,
                                                 validation_split=0.2,
                                                 subset="training",
                                                 seed=123)

val_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\kyley\Desktop\CS\C147\C147 Amazon Classification\img_uniform_split/train", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=BATCH_SIZE,
                                                 validation_split=0.2,
                                                 subset="validation",
                                                 seed=123)

test_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\kyley\Desktop\CS\C147\C147 Amazon Classification\img_uniform_split/test", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=BATCH_SIZE)

Found 89663 files belonging to 10 classes.
Using 71731 files for training.
Found 89663 files belonging to 10 classes.
Using 17932 files for validation.
Found 9966 files belonging to 10 classes.


In [4]:
def preprocess_data(X, Y):
  # input_images = X.astype('float32')
  X_p = tf.keras.applications.resnet50.preprocess_input(X)
  Y_p = tf.one_hot(Y, 10)
  return X_p, Y_p

# train_X = preprocess_image_input(training_images)
# valid_X = preprocess_image_input(validation_images)
train_ds = train_ds.map(preprocess_data)
val_ds = val_ds.map(preprocess_data)
test_ds = test_ds.map(preprocess_data)

In [5]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(384, 384, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(6,6))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(64,64,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  # model.compile(optimizer='SGD', 
  #               loss='sparse_categorical_crossentropy',
  #               metrics = ['accuracy'])
  model.compile('adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 384, 384, 3)      0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 12, 12, 2048)      23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              209817

In [11]:
print(model.layers[2])
model.layers[2].trainable = False

EPOCHS = 3
history = model.fit(train_ds, epochs=EPOCHS, validation_data = val_ds, batch_size=1)

loss, accuracy = model.evaluate(test_ds, batch_size=1)

Epoch 1/3


KeyboardInterrupt: 